In [1]:
import requests
import pandas as pd

# Function to check active Status of DAG
def get_active_status(dag_response, dagDict):
    dagDict['Active'] = dag_response['is_active']
    return dagDict

# Function to check paused Status of DAG
def get_paused_status(dag_response, dagDict):
    dagDict['Paused'] = dag_response['is_paused']
    return dagDict

# Function to check the last execution date of DAG
def get_execution_date(dag_response, dagDict):
    dagDict['Execution_Date'] = dag_response['last_parsed_time']
    return dagDict

# Function to move dag dictionary to DataFrame
def toDataFrame(dagList):
    dagDF = pd.DataFrame(dagList)
    return dagDF

# Airflow API endpoint for getting DAGs
api_endpoint = 'http://localhost:8080/api/v1/dags'

# Username and password for authentication
username = 'airflow'
password = 'airflow'

# Send a GET request with authentication
response = requests.get(api_endpoint, auth=(username, password))

dagList = []

# Check if the request was successful (status code 200)
if response.status_code == 200:
    dags = response.json()['dags']

    # Print the list of DAGs
    print("DAGs:")
    for dag in dags:
        dag_id = dag['dag_id']
        print(dag_id)
        
        dag_api_endpoint = 'http://localhost:8080/api/v1/dags/{}/dagRuns'.format(dag_id)
    
        # Send a GET request with authentication
        response = requests.get(dag_api_endpoint, auth=(username,password))
        
        # Check if connection was successful (status code = 200)
        if response.status_code == 200:
            dag_runs = response.json()['dag_runs']
            
            # Check if any DAG Run Exists
            if dag_runs:
                latest_dag_run = dag_runs[-1]
                state = latest_dag_run['state']
                dagDict = {
                    'DAG ID': dag_id,
                    'State': state,
                    'run_type': latest_dag_run['run_type'],
                    'execution_date': latest_dag_run['execution_date'].split("T")[0]
                }
                
                dag_api_response = requests.get(api_endpoint + '/' + dag_id, auth=(username, password))
                if dag_api_response.status_code == 200:
                    dag_response = dag_api_response.json()
                    dagDict = get_active_status(dag_response, dagDict)
                    dagDict = get_paused_status(dag_response, dagDict)
                    # dagDict = get_execution_date(dag_response, dagDict)
                
                dagList.append(dagDict)
else:
    print(f"Failed to retrieve DAGs. Status Code: {response.status_code}")

# Convert the list of dictionaries to DataFrame
dagDF = toDataFrame(dagList)
print(dagDF)


DAGs:
Branch_N_Trigger_DAG
dataset_consumes_1
dataset_consumes_1_and_2
dataset_consumes_1_never_scheduled
dataset_consumes_unknown_never_scheduled
dataset_produces_1
dataset_produces_2
example_bash_operator
example_branch_datetime_operator
example_branch_datetime_operator_2
example_branch_datetime_operator_3
example_branch_dop_operator_v3
example_branch_labels
example_branch_operator
example_branch_python_operator_decorator
example_complex
example_dag_decorator
example_external_task_marker_child
example_external_task_marker_parent
example_kubernetes_executor
example_local_kubernetes_executor
example_nested_branch_dag
example_passing_params_via_test_command
example_python_operator
example_short_circuit_decorator
example_short_circuit_operator
example_skip_dag
example_sla_dag
example_subdag_operator
example_task_group
example_task_group_decorator
example_time_delta_sensor_async
example_trigger_controller_dag
example_trigger_target_dag
example_weekday_branch_operator
example_xcom
example_